In [ ]:
# Convert CSV file into JSON data formatting
# 	1.	Y: The target variable indicating whether an anomaly was detected on a specific date 
# (1 for anomaly, 0 for no anomaly). This likely serves as the classification label for anomaly detection.



# 	1.	Y, Data: Likely represent the dependent variable (Y) and the date (Data) for time-series data.
# 	2.	XAU: Represents gold prices (commonly used symbol for gold in finance).
# 	3.	BGNL: Could stand for Bloomberg Generic Index, or a custom abbreviation.
# 	4.	ECSURPUS: Economic surplus (specific to the U.S. or another economy).
# 	5.	BDIY: Baltic Dry Index, a measure of shipping and trade activity.
# 	6.	CRY: CRB Commodity Index, tracking commodity prices.
# 	7.	DXY: U.S. Dollar Index, a measure of the dollar’s strength against other currencies.
# 	8.	JPY, GBP: Exchange rates for Japanese Yen and British Pound, respectively.
# 	9.	Cl1: WTI Crude Oil Futures (CL is the ticker for crude oil futures).
# 	10.	VIX: Volatility Index, measuring market volatility.
# 	11.	USGG30YR, GT10, USGG2YR, USGG3M, US0001M: U.S. Treasury yields for 30-year, 10-year, 2-year, 3-month, and 1-month bonds.
# 	12.	GTDEM30Y, GTDEM10Y, GTDEM2Y: German government bond yields (30-year, 10-year, 2-year).
# 	13.	EONIA: Euro Overnight Index Average, a benchmark for euro interbank lending.
# 	14.	GTITL30YR, GTITL10YR, GTITL2YR: Italian government bond yields (30-year, 10-year, 2-year).
# 	15.	GTJPY30YR, GTJPY10YR, GTJPY2YR: Japanese government bond yields (30-year, 10-year, 2-year).
# 	16.	GTGBP30Y, GTGBP20Y, GTGBP2Y: British government bond yields (30-year, 20-year, 2-year).
# 	17.	LUMSTRUU: Likely a Bloomberg index, e.g., Bloomberg US Mortgage Backed Securities Index.
# 	18.	LMBITR: Bloomberg Municipal Bond Index Total Return.
# 	19.	LUACTRUU: Bloomberg U.S. Corporate Aggregate Total Return Index.
# 	20.	LF98TRUU: Bloomberg Global High Yield Index.
# 	21.	LG30TRUU: Bloomberg Global Aggregate 30-Year Total Return Index.
# 	22.	LP01TREU: Likely Bloomberg Pan-European Corporate Total Return Index.
# 	23.	EMUSTRUU: Bloomberg Eurozone Aggregate Total Return Index.
# 	24.	LF94TRUU: Bloomberg Global Emerging Market Corporate Total Return Index.
# 	25.	MXUS, MXEU, MXJP, MXBR, MXRU, MXIN, MXCN: MSCI indices for countries/regions (U.S., Europe, Japan, Brazil, Russia, India, China).


In [ ]:
# Predict anomaly detection